# Kaggle Learn-and-compete `Poker Rule Induction`

## Data Description

You are provided with 25,010 poker hands in train.csv and 1,000,000 in test.csv. Each hand consists of five cards with a given suit and rank, drawn from a standard deck of 52. Suits and ranks are represented as ordinal categories:

```
S1 “Suit of card #1”
Ordinal (1-4) representing {Hearts, Spades, Diamonds, Clubs}
C1 “Rank of card #1”
Numerical (1-13) representing (Ace, 2, 3, ... , Queen, King)

...

S5 “Suit of card #5”
C5 “Rank of card #5”
```

Each row in the training set has the accompanying class label for the poker hand it comprises. The hands are omitted from the test set and must be predicted by participants. Hands are classified into the following ordinal categories:


```
0: Nothing in hand; not a recognized poker hand 
1: One pair; one pair of equal ranks within five cards
2: Two pairs; two pairs of equal ranks within five cards
3: Three of a kind; three equal ranks within five cards
4: Straight; five cards, sequentially ranked with no gaps
5: Flush; five cards with the same suit
6: Full house; pair + different rank three of a kind
7: Four of a kind; four equal ranks within five cards
8: Straight flush; straight + flush
9: Royal flush; {Ace, King, Queen, Jack, Ten} + flush
```
Note that the Straight flush and Royal flush hands are not representative of
the true domain because they have been over-sampled. The straight flush
is 14.43 times more likely to occur in the training set, while the royal flush is 129.82 times more likely.

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from functools import reduce

#### Read data

In [2]:
test_dataset = pd.read_csv('test.csv', delimiter=',')

In [3]:
test_dataset

,id,S1,C1,S2,C2,S3,C3,S4,C4,S5,C5
0,1,1,10,2,2,3,3,3,8,1,1
1,2,2,13,3,5,3,7,4,6,1,4
2,3,1,3,1,11,2,8,2,1,2,4
3,4,1,6,3,3,4,7,1,8,3,11
4,5,2,10,3,4,1,6,2,12,2,6
5,6,1,4,3,10,2,11,2,6,1,7
6,7,1,10,3,8,1,4,3,11,3,9
7,8,2,11,3,8,1,1,1,11,2,3
8,9,3,4,1,1,1,3,3,5,3,6
9,10,3,12,2,1,1,3,1,2,3,10


In [4]:
train_dataset = pd.read_csv('train.csv', delimiter=',')

In [5]:
data = train_dataset[['S1', 'C1','S2', 'C2','S3', 'C3','S4', 'C4','S5', 'C5']]
labels = train_dataset[['hand']]

In [6]:
data[0:10]

,S1,C1,S2,C2,S3,C3,S4,C4,S5,C5
0,4,9,2,1,2,2,4,7,2,8
1,1,4,3,6,1,12,3,11,2,7
2,1,11,4,1,3,7,4,11,2,1
3,2,9,2,4,3,6,1,9,4,9
4,1,8,2,4,2,11,2,2,2,1
5,2,5,1,5,2,13,2,3,3,13
6,3,10,4,6,1,4,2,13,4,5
7,4,10,3,1,2,13,4,2,4,7
8,3,2,4,10,3,3,4,4,1,9
9,2,7,3,8,4,8,2,13,2,12


In [7]:
labels_vect = tf.keras.utils.to_categorical(labels)

#### One hot encoding

In [8]:
def data_hotencode(data, columns, encoder, mode='transform'):
    encoded = None
    if (mode == 'fit'):
        encoded = encoder.fit_transform(data[columns])
    elif (mode == 'transform'):
        encoded = encoder.transform(data[columns])
    return pd.concat([data.drop(columns, axis=1), pd.DataFrame(encoded.toarray())], axis=1)

def hand_hotencode(hand, columns, encoder):
    df = pd.DataFrame(data=[hand], columns=['S1', 'C1','S2', 'C2','S3', 'C3','S4', 'C4','S5', 'C5'])
    return data_hotencode(df, columns, encoder, 'transform')

In [9]:
def shuffle_columns(data, columns):
    shuffled = data[columns].reindex(np.random.permutation(data[columns].columns), axis=1)
    return pd.concat([data.drop(columns, axis=1), shuffled], axis=1)

In [9]:
encoder = OneHotEncoder()
columns_to_hotencode = ['S1', 'S2', 'S3', 'S4', 'S5']
data_encoded = data_hotencode(data, columns_to_hotencode, encoder, 'fit')
print(data_encoded.shape)

data_encoded.head()

(25010, 25)


,C1,C2,C3,C4,C5,0,1,2,3,4,...,10,11,12,13,14,15,16,17,18,19
0,9,1,2,7,8,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,4,6,12,11,7,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,11,1,7,11,1,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,9,4,6,9,9,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,8,4,11,2,1,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


#### Sort

In [79]:
def sort(data, columns, ace_is_max = False):
    sorted = data[columns]
    if ace_is_max:
        sorted = sorted.replace(1, 14) - 1
    else:
        sorted = sorted
    
    sorted.values.sort()
    return pd.concat([sorted, data.drop(columns, axis=1)], axis=1)

In [80]:
columns_to_sort = ['C1', 'C2', 'C3', 'C4', 'C5']
data_sorted = sort(data_encoded, columns_to_sort)
data_sorted.head()

,C1,C2,C3,C4,C5,0,1,2,3,4,...,10,11,12,13,14,15,16,17,18,19
0,1,2,7,8,9,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,4,6,7,11,12,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,1,1,7,11,11,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,4,6,9,9,9,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1,2,4,8,11,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


#### Normalization

In [27]:
def normalize(data, columns, mean, std):
    data_normalized = (data[columns] - mean) / std
    to_return = pd.concat([data.drop(columns, axis=1), data_normalized], axis=1)
    to_return.columns = list(range(to_return.shape[1]))
    return to_return

In [83]:
columns_to_normalize = ['C1', 'C2', 'C3', 'C4', 'C5']
mean = data_sorted[columns_to_sort].mean().mean()
std = pd.Series(data_sorted[columns_to_sort].values.flatten()).std()
data_normalized = normalize(data_sorted, columns_to_normalize, mean, std)
data_normalized.head()

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,1.0,0.0,0.0,-1.596136,-1.329480,0.003798,0.270453,0.537109
1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,-0.796169,-0.262858,0.003798,1.070420,1.337076
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,-1.596136,-1.596136,0.003798,1.070420,1.070420
3,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,-0.796169,-0.262858,0.537109,0.537109,0.537109
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,-1.596136,-1.329480,-0.796169,0.270453,1.070420


#### 52 encoding

In [109]:
def data_52encode(data):
    columns = []
    for i in range(1, 5):
        for j in range(1, 14):
            columns.append(str(i) + str(j))
    
    data_squashed = pd.DataFrame()
    for i in range(1, 6):
        column_S = 'S'+str(i)
        column_C = 'C' + str(i)
        data_squashed[column_S+column_C] = data[column_S].astype(str) + data[column_C].astype(str)
    
    data_encoded = pd.DataFrame(data = np.zeros((data.shape[0], len(columns))), columns=columns)
    
    for i in range(data.shape[0]):
        row = data_squashed.iloc[i, :]
        for col in data_squashed.columns:
            data_encoded[row[col]][i] = 1
    
    return data_encoded

In [110]:
data_52encoded = data_52encode(data)

KeyboardInterrupt: 

In [111]:
def get_hand_df(hand):
    return pd.DataFrame({
        'S1': [hand[0]],
        'C1': [hand[1]],
        'S2': [hand[2]],
        'C2': [hand[3]],
        'S3': [hand[4]],
        'C3': [hand[5]],
        'S4': [hand[6]],
        'C4': [hand[7]],
        'S5': [hand[8]],
        'C5': [hand[9]],
    })

#### Test function

In [29]:
def calc_hand_label(hand):    
    def f(hand):
        suits_hist = reduce(lambda d,x: {**d, **{x: (d.get(x, 0) + 1)}}, hand[:,0], {})
        ranks_hist = reduce(lambda d,x: {**d, **{x: (d.get(x, 0) + 1)}}, hand[:,1], {})
        
        if len(ranks_hist.values()) < 5:
            if len(ranks_hist.values()) == 2:
                if max(list(ranks_hist.values())) == 4:
                    return 7
    #                 print('7: Four of a kind; four equal ranks within five cards')
                else:
                    return 6
    #                 print('6 Full house; pair + different rank three of a kind')
            elif len(ranks_hist.values()) == 3:
                if max(list(ranks_hist.values())) == 3:
                    return 3
    #                 print('3: Three of a kind; three equal ranks within five cards')
                else:
                    return 2
    #                 print('2: Two pairs; two pairs of equal ranks within five cards')
            else:
                return 1
    #             print('1: One pair; one pair of equal ranks within five cards')
        else:
            if len(suits_hist.values()) == 1:
                if max(list(ranks_hist.keys())) -  min(list(ranks_hist.keys())) == 4:
                    if max(list(ranks_hist.keys())) == 13:
                        return 9
    #                     print('9: Royal flush; {Ace, King, Queen, Jack, Ten} + flush')
                    else:
                        return 8
    #                     print('8: Straight flush; straight + flush')
                else:
                    return 5
    #                 print('5: Flush; five cards with the same suit')
            elif max(list(ranks_hist.keys())) -  min(list(ranks_hist.keys())) == 4:
                return 4    
    #             print('4: Straight; five cards, sequentially ranked with no gaps')
            else:
                return 0
                print('0: Nothing in hand; not a recognized poker hand ')
                
    hand = np.array(hand).reshape(5,2)
    res1 = f(hand)
    
    hand[:,1] = list(map(lambda x: 13 if (x == 1) else x-1, hand[:,1]))
    res2 = f(hand)
    
    if res1 == 9 and res2 == 8:
        return res2
    else:
        return max(res1, res2)

#### Encode cards

In [30]:
def card_encode(s, c):
    cards=["A","2","3","4","5","6","7","8","9","10","J","Q","K"]
    suits="♥♦♣♠"
    return cards[c - 1] + suits[s - 1]

def hand_encode(hand):
    return ', '.join(map(lambda x: card_encode(hand[x*2], hand[x*2+1]),range(5)))

print(hand_encode([4,9,2,1,2,2,4,7,2,8]))
print(hand_encode([2,9,2,4,3,6,1,9,4,9]))

9♠, A♦, 2♦, 7♠, 8♦
9♦, 4♦, 6♣, 9♥, 9♠


### Solutions

1. Neural network
2. Random forest

Ousiders:
 - Genetic algorithm
 - Decision tree

### Neural Network

In [31]:
def fit(model, data, labels, columns_to_shuffle, iterations, epochs, batch_size):
    for i in range(iterations):
        print ('-----------------------Iteration:' + str(i+1) +  '-----------------------')
        model.fit(np.array(data), np.array(labels), epochs=epochs, batch_size=batch_size)
        data = shuffle_columns(data, columns_to_shuffle)
    return model

In [82]:
from time import time
from keras.callbacks import TensorBoard

model = keras.Sequential()
#input
model.add(keras.layers.Dense(64, activation='relu', input_shape=(25,)))
model.add(keras.layers.Dropout(0.1))
#hidden
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dropout(0.05))
model.add(keras.layers.Dense(16, activation='relu'))
# Add a softmax layer with 10 output units:
model.add(keras.layers.Dense(10, activation='softmax'))

model.compile(optimizer=tf.train.AdamOptimizer(0.0005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

tensorboard = TensorBoard(log_dir="logs/{}".format(time()))

model.fit(np.array(data_normalized), np.array(labels_vect), epochs=1000, batch_size=512)

Epoch 1/1000
25010/25010 [==============================]25010/25010 [==============================] - 1s 41us/step - loss: 2.0189 - acc: 0.3002

Epoch 2/1000
25010/25010 [==============================]25010/25010 [==============================] - 0s 10us/step - loss: 1.3037 - acc: 0.4954

Epoch 3/1000
25010/25010 [==============================]25010/25010 [==============================] - 0s 10us/step - loss: 1.0898 - acc: 0.4908

Epoch 4/1000
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 1.0461 - acc: 0.5039

Epoch 5/1000
25010/25010 [==============================]25010/25010 [==============================] - 0s 10us/step - loss: 1.0224 - acc: 0.5083

Epoch 6/1000
25010/25010 [==============================]25010/25010 [==============================] - 0s 9us/step - loss: 1.0050 - acc: 0.5176

Epoch 7/1000
25010/25010 [==============================]25010/25010 [==============================] - 0s 9us/step - lo

25010/25010 [==============================]25010/25010 [==============================] - 0s 9us/step - loss: 0.9411 - acc: 0.5524

Epoch 57/1000
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 0.9418 - acc: 0.5546

Epoch 58/1000
25010/25010 [==============================]25010/25010 [==============================] - 0s 9us/step - loss: 0.9416 - acc: 0.5533

Epoch 59/1000
25010/25010 [==============================]25010/25010 [==============================] - 0s 9us/step - loss: 0.9402 - acc: 0.5521

Epoch 60/1000
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 0.9412 - acc: 0.5528

Epoch 61/1000
25010/25010 [==============================]25010/25010 [==============================] - 0s 9us/step - loss: 0.9404 - acc: 0.5544

Epoch 62/1000
25010/25010 [==============================]25010/25010 [==============================] - 0s 9us/step - loss: 0.9410 

25010/25010 [==============================]25010/25010 [==============================] - 0s 9us/step - loss: 0.9155 - acc: 0.5704

Epoch 112/1000
25010/25010 [==============================]25010/25010 [==============================] - 0s 9us/step - loss: 0.9150 - acc: 0.5693

Epoch 113/1000
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 0.9151 - acc: 0.5701

Epoch 114/1000
25010/25010 [==============================]25010/25010 [==============================] - 0s 9us/step - loss: 0.9145 - acc: 0.5720

Epoch 115/1000
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 0.9131 - acc: 0.5701

Epoch 116/1000
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 0.9134 - acc: 0.5718

Epoch 117/1000
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 0

25010/25010 [==============================]25010/25010 [==============================] - 0s 10us/step - loss: 0.8887 - acc: 0.5855

Epoch 167/1000
25010/25010 [==============================]25010/25010 [==============================] - 0s 9us/step - loss: 0.8886 - acc: 0.5843

Epoch 168/1000
25010/25010 [==============================]25010/25010 [==============================] - 0s 9us/step - loss: 0.8849 - acc: 0.5885

Epoch 169/1000
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 0.8841 - acc: 0.5889

Epoch 170/1000
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 0.8847 - acc: 0.5888

Epoch 171/1000
25010/25010 [==============================]25010/25010 [==============================] - 0s 9us/step - loss: 0.8842 - acc: 0.5870

Epoch 172/1000
25010/25010 [==============================]25010/25010 [==============================] - 0s 9us/step - loss: 

25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 0.8718 - acc: 0.5979

Epoch 222/1000
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 0.8720 - acc: 0.5958

Epoch 223/1000
25010/25010 [==============================]25010/25010 [==============================] - 0s 9us/step - loss: 0.8702 - acc: 0.5969

Epoch 224/1000
25010/25010 [==============================]25010/25010 [==============================] - 0s 9us/step - loss: 0.8726 - acc: 0.5965

Epoch 225/1000
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 0.8701 - acc: 0.5946

Epoch 226/1000
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 0.8703 - acc: 0.5989

Epoch 227/1000
25010/25010 [==============================]25010/25010 [==============================] - 0s 9us/step - loss: 0

...

25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 0.8271 - acc: 0.6171

Epoch 607/1000
25010/25010 [==============================]25010/25010 [==============================] - 0s 9us/step - loss: 0.8234 - acc: 0.6221

Epoch 608/1000
25010/25010 [==============================]25010/25010 [==============================] - 0s 9us/step - loss: 0.8229 - acc: 0.6202

Epoch 609/1000
25010/25010 [==============================]25010/25010 [==============================] - 0s 9us/step - loss: 0.8232 - acc: 0.6209

Epoch 610/1000
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 0.8246 - acc: 0.6186

Epoch 611/1000
25010/25010 [==============================]25010/25010 [==============================] - 0s 9us/step - loss: 0.8277 - acc: 0.6175

Epoch 612/1000
25010/25010 [==============================]25010/25010 [==============================] - 0s 9us/step - loss: 0

25010/25010 [==============================]25010/25010 [==============================] - 0s 9us/step - loss: 0.8242 - acc: 0.6189

Epoch 662/1000
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 0.8236 - acc: 0.6230

Epoch 663/1000
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 0.8239 - acc: 0.6206

Epoch 664/1000
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 0.8209 - acc: 0.6218

Epoch 665/1000
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 0.8216 - acc: 0.6224

Epoch 666/1000
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 0.8245 - acc: 0.6206

Epoch 667/1000
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 0

25010/25010 [==============================]25010/25010 [==============================] - 0s 10us/step - loss: 0.8131 - acc: 0.6269

Epoch 717/1000
25010/25010 [==============================]25010/25010 [==============================] - 0s 9us/step - loss: 0.8140 - acc: 0.6273

Epoch 718/1000
25010/25010 [==============================]25010/25010 [==============================] - 0s 10us/step - loss: 0.8163 - acc: 0.6253

Epoch 719/1000
25010/25010 [==============================]25010/25010 [==============================] - 0s 11us/step - loss: 0.8179 - acc: 0.6223

Epoch 720/1000
25010/25010 [==============================]25010/25010 [==============================] - 0s 10us/step - loss: 0.8133 - acc: 0.6277

Epoch 721/1000
25010/25010 [==============================]25010/25010 [==============================] - 0s 9us/step - loss: 0.8153 - acc: 0.6264

Epoch 722/1000
25010/25010 [==============================]25010/25010 [==============================] - 0s 9us/step - los

25010/25010 [==============================]25010/25010 [==============================] - 0s 10us/step - loss: 0.8151 - acc: 0.6229

Epoch 772/1000
25010/25010 [==============================]25010/25010 [==============================] - 0s 10us/step - loss: 0.8133 - acc: 0.6275

Epoch 773/1000
25010/25010 [==============================]25010/25010 [==============================] - 0s 10us/step - loss: 0.8138 - acc: 0.6258

Epoch 774/1000
25010/25010 [==============================]25010/25010 [==============================] - 0s 9us/step - loss: 0.8166 - acc: 0.6257

Epoch 775/1000
25010/25010 [==============================]25010/25010 [==============================] - 0s 9us/step - loss: 0.8134 - acc: 0.6282

Epoch 776/1000
25010/25010 [==============================]25010/25010 [==============================] - 0s 9us/step - loss: 0.8147 - acc: 0.6272

Epoch 777/1000
25010/25010 [==============================]25010/25010 [==============================] - 0s 10us/step - los

KeyboardInterrupt: 

In [33]:
y_test = test_dataset.drop(['id'], axis=1).apply(calc_hand_label, axis=1)

In [39]:
model.save('./model-sort-166')

### Predictions

In [34]:
test_dataset_noid = test_dataset.drop(['id'], axis=1)

In [66]:
test_sample = test_dataset_noid
test_data_sample_encoded_1 = normalize(
    sort(
        data_hotencode(test_sample, columns_to_hotencode, encoder),
        columns_to_sort,
        False,
    ), 
    columns_to_normalize,
    mean,
    std
)

test_data_sample_encoded_2 = normalize(
    sort(
        data_hotencode(test_sample, columns_to_hotencode, encoder),
        columns_to_sort,
        True,
    ), 
    columns_to_normalize,
    mean,
    std
)

In [56]:
model1 = keras.models.load_model('./model-sort-166')
model1.compile(optimizer=tf.train.AdamOptimizer(0.0005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model2 = keras.models.load_model('./model-sort-174')
model2.compile(optimizer=tf.train.AdamOptimizer(0.0005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [6]:
test_hand = [1,9,1,10,1,11,1,12,1,1]
print(hand_encode(test_hand))
prediction = model.predict(np.array(normalize(hand_hotencode(test_hand, columns_to_hotencode, encoder), columns_to_normalize, scaler)))
print(prediction[0])
plt.plot(prediction[0])

9♥, 10♥, J♥, Q♥, A♥


NameError: name 'model' is not defined

In [87]:
test_data_sample_predicted_1_10 = model1.predict(np.array(test_data_sample_encoded_1))
test_data_sample_predicted_1 = np.array(list(map(np.argmax, test_data_sample_predicted_1_10)))
test_data_sample_predicted_probs_1 = np.array(list(map(np.max, test_data_sample_predicted_1_10)))
test_data_sample_predicted_2_10 = model2.predict(np.array(test_data_sample_encoded_2))
test_data_sample_predicted_2 = np.array(list(map(np.argmax, test_data_sample_predicted_2_10)))
test_data_sample_predicted_probs_2 = np.array(list(map(np.max, test_data_sample_predicted_2_10)))

In [103]:
preds = np.concatenate(
    (
        test_data_sample_predicted_1.reshape(len(test_data_sample_predicted_1), 1),
        test_data_sample_predicted_2.reshape(len(test_data_sample_predicted_2), 1)
    ),
    axis=1
)

probs = np.concatenate(
    (
        test_data_sample_predicted_probs_1.reshape(len(test_data_sample_predicted_probs_1), 1),
        test_data_sample_predicted_probs_2.reshape(len(test_data_sample_predicted_probs_2), 1)
    ),
    axis=1
)

indices = np.array(list(map(np.argmax, probs))).reshape(1000000,1)

In [104]:
indices.shape

(1000000, 1)

In [114]:
test_data_sample_predicted = []
for p, index in zip(preds, indices):
    test_data_sample_predicted.append(p[index[0]])

In [116]:
#test_data_sample_predicted = np.maximum(test_data_sample_predicted_1, test_data_sample_predicted_2)
#test_data_sample_predicted = test_data_sample_predicted_2

test_data_sample_labels = y_test
df = pd.DataFrame({
    'predicted_1': test_data_sample_predicted_1,
    'predicted_2': test_data_sample_predicted_2,
    'predicted_probs_1': test_data_sample_predicted_probs_1,
    'predicted_probs_2': test_data_sample_predicted_probs_2,
    'predicted': test_data_sample_predicted, 
    'real': test_data_sample_labels
})
df = pd.concat([test_sample, df], axis=1)
wrong_answers = df[df['predicted'] != df['real']]
print(wrong_answers.shape)
wrong_answers

(24, 16)


,S1,C1,S2,C2,S3,C3,S4,C4,S5,C5,predicted_1,predicted_2,predicted_probs_1,predicted_probs_2,predicted,real
14657,4,1,1,5,3,4,3,3,1,2,4,0,0.996941,0.999503,0,4
153085,4,4,1,1,4,2,1,3,1,5,4,0,0.997092,0.998551,0,4
189896,2,12,4,12,2,11,3,12,1,12,6,6,0.843938,0.700288,6,7
325202,2,8,2,9,3,9,4,9,1,9,3,7,0.911571,0.635526,3,7
333453,1,11,1,10,2,11,4,11,3,11,6,6,0.542889,0.630375,6,7
473784,3,3,4,5,3,1,3,2,3,4,4,0,0.991499,0.997622,0,4
511090,1,13,3,13,3,11,4,13,2,13,3,6,0.470374,0.514752,6,7
519829,3,1,3,2,4,3,3,4,3,5,4,0,0.991140,0.999485,0,4
561476,4,11,3,11,1,10,2,11,1,11,6,7,0.982283,0.537851,6,7
583292,4,5,1,4,3,3,1,1,2,2,4,0,0.998962,0.999341,0,4


In [109]:
wrong_answers['real'].value_counts()

7    15
4     8
8     1
Name: real, dtype: int64

In [117]:
wrong_answers.drop(['predicted_1', 'predicted_2', 'predicted_probs_1', 'predicted_probs_2', 'predicted', 'real'], axis=1).apply(lambda h: hand_encode(np.array(h)), axis=1)

14657      A♠, 5♥, 4♣, 3♣, 2♥
153085     4♠, A♥, 2♠, 3♥, 5♥
189896     Q♦, Q♠, J♦, Q♣, Q♥
325202     8♦, 9♦, 9♣, 9♠, 9♥
333453    J♥, 10♥, J♦, J♠, J♣
473784     3♣, 5♠, A♣, 2♣, 4♣
511090     K♥, K♣, J♣, K♠, K♦
519829     A♣, 2♣, 3♠, 4♣, 5♣
561476    J♠, J♣, 10♥, J♦, J♥
583292     5♠, 4♥, 3♣, A♥, 2♦
585044     A♥, A♦, A♣, A♠, K♣
628440     9♦, 6♠, 9♣, 9♠, 9♥
640832     K♣, K♥, K♦, K♠, Q♣
654165     K♣, K♥, K♠, K♦, Q♦
691555     5♠, A♥, 4♠, 2♥, 3♥
692475     Q♦, Q♠, Q♥, 9♠, Q♣
723435     A♠, 2♣, 5♣, 4♣, 3♠
763760     3♠, 2♣, A♣, 5♣, 4♠
819607     Q♣, Q♥, Q♠, 9♦, Q♦
845678     K♦, J♠, K♣, K♠, K♥
888126     9♥, 9♦, 9♣, 9♠, 8♠
912142    10♠, Q♠, 9♠, J♠, K♠
925292     K♦, K♥, K♠, Q♠, K♣
932343     7♥, 7♦, 7♠, 6♠, 7♣
dtype: object